## 1. Import Libraries 

In [ ]:
# Import libraries TensorFlow
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.datasets import mnist

In [ ]:
# Import basic numpy
import numpy as np
from itertools import product

In [ ]:
# Import Sklern libs
import sklearn
from sklearn.metrics import roc_auc_score

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from random import random, sample

In [ ]:
# Import MLflow 
import mlflow
import mlflow.tensorflow

## 2. Load dataset and preprocessing

In [ ]:
# Load dataset MNIST end split in train test
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
print("Size train: {} ... Size test: {}".format(len(y_train),len(y_test)))

In [ ]:
num_row = 2
num_col = 5 # plot images
fig, axes = plt.subplots(num_row, num_col, figsize=(1.5*num_col,2*num_row))
for i in range(10):
    ax = axes[i//num_col, i%num_col]
    ax.imshow(x_train[i], cmap='gray')
    ax.set_title('Label: {}'.format( y_train[i]))
plt.tight_layout()
plt.show()

In [ ]:
print("Shape x_train: {} ... Size x_test: {}".format(x_train.shape,x_test.shape))

In [ ]:
# Since 2D convolutional layers in TensorFlow/Keras expect four 
# dimensions in the format of (m, h, w, c) where m stands for the number of
# samples in the dataset, h and w stand for the height and width, respectively,
# and c stands for the number of channels (three if it’s an RGB color image for example),

In [ ]:
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1],
x_train.shape[2], 1)

In [ ]:
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1],
x_test.shape[2], 1)

In [ ]:
print("Shape x_train: {} ... Size x_test: {}".format(x_train.shape,x_test.shape))

In [ ]:
# Transformate to categorical (basic one hot encoder)
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

## 3. Modeling - CNN 

In [ ]:
# Function to generate (custom) CNN.
def generateModelCnn(funcActivation,nFilter):
    
    model = models.Sequential()
    model.add(layers.Conv2D(nFilter, (3, 3), activation=funcActivation, input_shape=(28, 28, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(nFilter, (3, 3), activation=funcActivation))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(nFilter, (3, 3), activation=funcActivation))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation=funcActivation))
    model.add(layers.Dense(10,activation='softmax'))
    model.summary()
    
    return model

In [ ]:
# Creating a basic dictionary for search hyperparameters.
listFuncActivation = ['relu','tanh']
nFilter = [64,32]
dictParameters = [dict(zip(('activateFunc','Nfilter'), (i,j))) for i,j in product(listFuncActivation,nFilter)]

In [ ]:
dictParameters

## 4. MLflow - Experiment

In [ ]:
# provide uri of your tracking server (e.g http://127.0.0.1:5000/ )
mlflow.set_tracking_uri('http://127.0.0.1:5000/')

In [ ]:
mlflow.set_experiment("TF_Keras_MNIST_Example")

In [ ]:
for Parameters in dictParameters:
    
    model = generateModelCnn(Parameters['activateFunc'],Parameters['Nfilter'])
    model.compile(optimizer="Adam",
                  loss="categorical_crossentropy", metrics=["accuracy"])
    
    with mlflow.start_run():

        mlflow.tensorflow.autolog()
        model.fit(x=x_train, y=y_train, batch_size=512, epochs=10)
        preds = model.predict(x_test)
        preds = np.round(preds)
        eval_acc = model.evaluate(x_test, y_test)[1]
        auc_score = roc_auc_score(y_test, preds)
        print("eval_acc: ", eval_acc)
        print("auc_score: ", auc_score)
        mlflow.tensorflow.mlflow.log_metric("eval_acc", eval_acc)
        mlflow.tensorflow.mlflow.log_metric("auc_score", auc_score)


    mlflow.end_run()

In [ ]:
mlflow.tensorflow.autolog()

## 5. MLflow - Load Model 

In [ ]:
dataframeRuns = mlflow.search_runs()

In [ ]:
dataframeRuns.sort_values(['metrics.auc_score'],inplace=False,ascending =  True)

In [ ]:
bestModel = dataframeRuns.iloc[0]

In [ ]:
bestModel

In [ ]:
# Example...
logged_model = 'runs:/'+bestModel.run_id+'/model'

In [ ]:
loaded_model = mlflow.keras.load_model(logged_model)

In [ ]:
eval_loss, eval_acc = loaded_model.evaluate(x_test, y_test)
preds = loaded_model.predict(x_test)
preds = np.round(preds)
eval_auc = roc_auc_score(y_test, preds)

In [ ]:
print("Eval Loss:", eval_loss)
print("Eval Acc:", eval_acc)
print("Eval AUC:", eval_auc)

In [ ]:
#mlflow server --backend-store-uri="sqlite:///mlflow.db"  --default-artifact-root="tracks\\"

In [ ]:
bestModel['artifact_uri']

## 6. MLflow - Using the model in production (local)

In [ ]:
# mlflow models serve -m {PATH_MODEL} -p 1234

In [ ]:
import requests 
import json 

In [ ]:
exampleImg = x_test[0]

In [ ]:
endpoint = "http://localhost:1234/invocations" 
headers = {"Content-type": "application/json; format=pandas-split"} 
#response = requests.post(endpoint, json = json.loads(test) , headers=headers)

In [ ]:
response